In [2]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpInteger
from openpyxl import load_workbook
import pandas as pd
import json

# Load the data from CSV files
camp_data = pd.read_csv('/home/roniya/CNAID/LP/camp.csv')
station_data = pd.read_csv('/home/roniya/CNAID/LP/station.csv')
vehicle_data = pd.read_csv('/home/roniya/CNAID/LP/vechicle.csv')

# Prepare the demand data
demand = {}
for index, row in camp_data.iterrows():
    camp = row['To']
    demand[camp] = row['demand_kg']

# Prepare the distance data
distance = {}
for index, row in camp_data.iterrows():
    station = row['From']
    camp = row['To']
    distance[(station, camp)] = row['distance_km']

# Parse the station data
stations = {}
for index, row in station_data.iterrows():
    station = row['station']
    capacity = row['capacity']
    # Convert JSON-like string to a dictionary
    mode_of_transport = json.loads(row['mode_of_transport'])
    stations[station] = {
        'capacity': capacity,
        'modes': mode_of_transport
    }

# Parse the vehicle data
vehicle_costs = {}
for index, row in vehicle_data.iterrows():
    mode = row['mode']
    vehicle_costs[mode] = {
        'capacity': row['capacity'],
        'total_cost_per_km': row['total_cost_per_km']
    }

# Define the LP problem
prob = LpProblem("Minimize_Transportation_Cost", LpMinimize)

# Decision Variables (ensure integer values for trips)
x = LpVariable.dicts("Trips", (stations.keys(), demand.keys(), vehicle_costs.keys()), lowBound=0, cat=LpInteger)

# Objective Function: Minimize total transportation cost
prob += lpSum(x[i][j][k] * vehicle_costs[k]['total_cost_per_km'] * distance.get((i, j), 0)
               for i in stations.keys() for j in demand.keys() if (i, j) in distance for k in vehicle_costs.keys()), "Total_Cost"

# Constraints

# Demand satisfaction at each camp
for j in demand.keys():
    prob += lpSum(x[i][j][k] * vehicle_costs[k]['capacity'] for i in stations.keys() if (i, j) in distance for k in vehicle_costs.keys()) >= demand[j], f"Demand_{j}"

# Capacity constraints
for i in stations.keys():
    for j in demand.keys():
        if (i, j) in distance:
            for k in vehicle_costs.keys():
                if k in stations[i]['modes']:
                    prob += x[i][j][k] * vehicle_costs[k]['capacity'] >= 0, f"Capacity_{i}_{j}_{k}"

# Vehicle limits constraints
for i in stations.keys():
    for k in vehicle_costs.keys():
        if k in stations[i]['modes']:
            max_trips = stations[i]['modes'][k].get('Max Number of Trips Allowed', float('inf'))
            num_vehicles = stations[i]['modes'][k].get('Number of Vehicles Available', 0)
            prob += lpSum(x[i][j][k] for j in demand.keys() if (i, j) in distance) <= max_trips * num_vehicles, f"Vehicle_Limit_{i}_{k}"

# Solve the problem
prob.solve()

# Output results
results = []
total_transportation_cost = 0

for i in stations.keys():
    for j in demand.keys():
        for k in vehicle_costs.keys():
            trips = x[i][j][k].varValue
            if trips and trips > 0:
                quantity_transferred = trips * vehicle_costs[k]['capacity']
                cost = trips * vehicle_costs[k]['total_cost_per_km'] * distance.get((i, j), 0)
                total_transportation_cost += cost
                results.append({
                    'Station': i,
                    'Camp': j,
                    'Vehicle': k,
                    'Number_of_Trips': int(trips),
                    'Demand_of_Camp': demand[j],
                    'Amount_Transferred': quantity_transferred,
                    'Total_Cost': cost
                })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Calculate total amount transferred to each camp
total_transferred_per_camp = results_df.groupby(['Station', 'Camp'])['Amount_Transferred'].sum().reset_index()
total_transferred_per_camp.rename(columns={'Amount_Transferred': 'Total_Amount_Transferred'}, inplace=True)

# Calculate total cost for each station-camp pair
total_cost_per_pair = results_df.groupby(['Station', 'Camp'])['Total_Cost'].sum().reset_index()
total_cost_per_pair.rename(columns={'Total_Cost': 'Total_Cost_Per_Station_Camp'}, inplace=True)

# Calculate total cost per station
total_cost_per_station = results_df.groupby('Station')['Total_Cost'].sum().reset_index()
total_cost_per_station.rename(columns={'Total_Cost': 'Total_Cost_Per_Station'}, inplace=True)

# Merge the aggregated totals into results_df
results_df = pd.merge(results_df, total_transferred_per_camp, on=['Station', 'Camp'])
results_df = pd.merge(results_df, total_cost_per_pair, on=['Station', 'Camp'])

# Arrange the columns as specified
results_df = results_df[['Station', 'Camp', 'Vehicle', 'Number_of_Trips', 'Demand_of_Camp', 'Amount_Transferred', 'Total_Amount_Transferred', 'Total_Cost', 'Total_Cost_Per_Station_Camp']]

# Save results to an Excel file using openpyxl
excel_filename = '/home/roniya/CNAID/LP/transportation_results.xlsx'
results_df.to_excel(excel_filename, index=False, engine='openpyxl')

# Load the workbook and select the active worksheet
wb = load_workbook(excel_filename)
ws = wb.active

# Merge cells for the 'Station' column
station_col_index = results_df.columns.get_loc('Station') + 1
last_row = ws.max_row
current_station = None
start_row = None

for row in range(2, last_row + 1):
    cell_value = ws.cell(row=row, column=station_col_index).value
    if cell_value != current_station:
        if start_row is not None and start_row < row - 1:
            ws.merge_cells(start_row=start_row, start_column=station_col_index, end_row=row - 1, end_column=station_col_index)
        current_station = cell_value
        start_row = row

# Merge cells for the last set of stations
if start_row is not None and start_row < last_row:
    ws.merge_cells(start_row=start_row, start_column=station_col_index, end_row=last_row, end_column=station_col_index)

# Merge cells for the 'Camp' column
camp_col_index = results_df.columns.get_loc('Camp') + 1
current_camp = None
start_row = None

for row in range(2, last_row + 1):
    cell_value = ws.cell(row=row, column=camp_col_index).value
    if cell_value != current_camp:
        if start_row is not None and start_row < row - 1:
            ws.merge_cells(start_row=start_row, start_column=camp_col_index, end_row=row - 1, end_column=camp_col_index)
        current_camp = cell_value
        start_row = row

# Merge cells for the last set of camps
if start_row is not None and start_row < last_row:
    ws.merge_cells(start_row=start_row, start_column=camp_col_index, end_row=last_row, end_column=camp_col_index)

# Merge cells for the 'Demand_of_Camp' column
demand_col_index = results_df.columns.get_loc('Demand_of_Camp') + 1
current_demand = None
start_row = None

for row in range(2, last_row + 1):
    cell_value = ws.cell(row=row, column=demand_col_index).value
    if cell_value != current_demand:
        if start_row is not None and start_row < row - 1:
            ws.merge_cells(start_row=start_row, start_column=demand_col_index, end_row=row - 1, end_column=demand_col_index)
        current_demand = cell_value
        start_row = row

# Merge cells for the last set of demands
if start_row is not None and start_row < last_row:
    ws.merge_cells(start_row=start_row, start_column=demand_col_index, end_row=last_row, end_column=demand_col_index)

# Merge cells for the 'Total_Amount_Transferred' column
total_amount_col_index = results_df.columns.get_loc('Total_Amount_Transferred') + 1
current_total_amount = None
start_row = None

for row in range(2, last_row + 1):
    cell_value = ws.cell(row=row, column=total_amount_col_index).value
    if cell_value != current_total_amount:
        if start_row is not None and start_row < row - 1:
            ws.merge_cells(start_row=start_row, start_column=total_amount_col_index, end_row=row - 1, end_column=total_amount_col_index)
        current_total_amount = cell_value
        start_row = row

# Merge cells for the last set of total amounts
if start_row is not None and start_row < last_row:
    ws.merge_cells(start_row=start_row, start_column=total_amount_col_index, end_row=last_row, end_column=total_amount_col_index)

# Merge cells for the 'Total_Cost_Per_Station_Camp' column
total_cost_col_index = results_df.columns.get_loc('Total_Cost_Per_Station_Camp') + 1
current_total_cost = None
start_row = None

for row in range(2, last_row + 1):
    cell_value = ws.cell(row=row, column=total_cost_col_index).value
    if cell_value != current_total_cost:
        if start_row is not None and start_row < row - 1:
            ws.merge_cells(start_row=start_row, start_column=total_cost_col_index, end_row=row - 1, end_column=total_cost_col_index)
        current_total_cost = cell_value
        start_row = row

# Merge cells for the last set of total costs
if start_row is not None and start_row < last_row:
    ws.merge_cells(start_row=start_row, start_column=total_cost_col_index, end_row=last_row, end_column=total_cost_col_index)

# Add a total cost row
ws.append([])
ws.append(['', 'Total Cost Per Station'])
for row in range(len(total_cost_per_station)):
    ws.append([total_cost_per_station.iloc[row]['Station'], total_cost_per_station.iloc[row]['Total_Cost_Per_Station']])
    
# Add a total cost row for all stations
ws.append([])
ws.append(['', 'Overall Total Cost', total_transportation_cost])

# Save the workbook
wb.save(excel_filename)

print(f"Results have been saved and formatted in {excel_filename}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/roniya/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/32816ada93c24b7fa8fdb7a597221076-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/32816ada93c24b7fa8fdb7a597221076-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 550 COLUMNS
At line 5843 RHS
At line 6389 BOUNDS
At line 7524 ENDATA
Problem MODEL has 545 rows, 1134 columns and 1890 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 176132 - 0.00 seconds
Cgl0003I 0 fixed, 662 tightened bounds, 125 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0004I processed model has 167 rows, 1040 columns (1040 integer (18 of which binary)) and 1418 elements
Cbc0012I Integer solution of 496278.9 found by DiveCoefficient after 493 iteratio